In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import Code
import sys
from glob import glob
%matplotlib inline
from os.path import join as jn
import os
from BabelViscoFDTD.H5pySimple import ReadFromH5py, SaveToH5py
import numpy as np

# Run all `RunPCA_FIT_ZTE_PETRA.ipynb` steps for multiple cases
We will use the location of `Target.txt` files to identify the casez

In [ ]:
Ldirs=glob('**/**/Target.txt')
Ldirs.sort()
Ldirs

In [ ]:
for l in Ldirs:
    
    BasePath=os.path.split(l)[0]
    print('*'*60)
    print('*'*60)
    print(BasePath)
    print('*'*60)
    print('*'*60)
    

    simbnibs_path=jn(BasePath,'m2m_'+os.path.split(BasePath)[1])
    Mat4Trajectory=jn(BasePath,'Target.txt')
    ComputingDevice='M3'
    ID = 'Target'
    
    ZTE=jn(BasePath,'ZTE.nii.gz') #use always ZTE.nii.gz as file name regardless if ZTE o PETRA
    
    CTType =2 # 2 = ZTE, 3 = PETRA ; CHANGE accordingly
    
    # We instantiate the class
    P=Code.Processing(BasePath,
                          ID,
                          simbnibs_path,
                          Mat4Trajectory,
                          ComputingDevice=ComputingDevice,
                          ZTE=ZTE,
                          CTType=CTType)
    P.Run_mask_creation(bForceRecalculate=False)
    P.RunElastix('FiniteDifferenceGradientDescent',bCToMRI=True)
    P.ShowFirstPart(LineSep=120)
    P.ProcessMRI(ZTERange=[0.1,0.65])
    plt.close('all')

# Combine all ZTE/PETRA and CT data for a global PCA fitting

In [ ]:
AllRes=glob('**/**/PCAFitData.h5')
AllRes.sort()
AllRes

In [ ]:

for n,l in enumerate(AllRes):
    d=ReadFromH5py(l)
    if n==0:
        data=d['data']
        NewPCAFit=d['NewPCAFit']
    else:
        data=np.vstack((data,d['data']))
        NewPCAFit=np.vstack((NewPCAFit,d['NewPCAFit']))
    

In [ ]:
# print individualized PCA fit parameters
for e in NewPCAFit:
    print(e)

# Run PCA fitting

In [ ]:
# Run
bIsPetra = False # If false, we assume ZTE

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

sc = StandardScaler()

X=sc.fit_transform(data)
X_mean = np.mean(X, axis=0)

# Apply PCA
pca = PCA(n_components=2)
pca.fit(X)


'''
We get the PCA first component to obtain a linear fitting function
'''

# First principal component
pc1 = pca.components_[0]

# Compute slope (b/a)
m = pc1[1] / pc1[0]  # slope of the first principal component

# Compute intercept
c = X_mean[1] - m * X_mean[0]

# Generate fitted line points
x_fit = np.linspace(min(X[:,0]), max(X[:,0]), 100)
y_fit = m * x_fit + c

# Print the final line equation
xf2=x_fit*sc.scale_[0]+sc.mean_[0]
yf2=y_fit*sc.scale_[1]+sc.mean_[1]

# These are the conversion factors used by default in BabelBrain
if bIsPetra:
    yf3=xf2*(-2929.6)+ 3274.9
else:
    yf3=xf2*(-2085)+ 2329.0

In [ ]:
plt.figure()
plt.hist2d(data[:,0],data[:,1],bins=50)
plt.plot(xf2,yf2,label="PCA Fit", color='blue', linestyle='dashed')
plt.plot(xf2,yf3,label="Orig.", color='red', linestyle='dashed')
plt.xlabel("ZTE")
plt.ylabel("HY")
plt.legend()
plt.title("PCA-Based unnormalized Linear Fit")


'''
We convert from PCA normalized condition fitting to un-normalized
This is the new fitting formula for ZTE/PETRA to pseudo CT
'''
lin_fit=np.polyfit(xf2,yf2,1)
print(' New PCA fitting formula', lin_fit)